In [19]:
# ** = Change location if needed

In [18]:
# Seting up the environment:
# Reunite all the text file with the news in a folder called 'Analysis'
# Set up the basic path

basic_path = '/root/Desktop/' # **

path = basic_path + 'Analysis'

folders = ['/Il Corriere della Sera'] 
nick_names = ['/Corr']

choose = 0 # chose the number that represent the journal

actual_journal = [path +  folders[choose], nick_names[choose]]
print(actual_journal)

['/root/Desktop/Analysis/Il Corriere della Sera', '/Corr']


In [20]:
# Complete Analysis
# -----------------------------
# Import library

import re
import ast
import json
import nltk
import time
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

In [21]:
# Data loading

data = []
with open(actual_journal[0] + '.txt') as f:
    for line in f:
        data.append(json.loads(line, encoding='utf8'))
        print(line)

        
collection_testi = []
for i in range(0, len(data)):
    if len(str(data[i]['text'])) > 10:  # Delete the news with no text
        collection_testi.append(str(data[i]['title']) + str(data[i]['text']) + str(data[i]['tags']) + str(data[i]['resume']))
        
    

{"_id":{"$oid":"5a101c5d5fbf0004f5a4adc4"},"title":["\n                ","\n            "," Di Matteo: mio figlio finito nell’acido, Riina doveva morire 50 anni fa"],"text":"\n ROMA «È morto troppo tardi, doveva morire cinquant’anni fa», mormora nel suo incorreggibile dialetto siciliano Santino Di Matteo, il pentito di mafia che ha confessato la strage di Capaci e al quale Cosa nostra, dopo le sue prime dichiarazioni ai magistrati, ha rapito il figlio tredicenne Giuseppe, ucciso dopo due anni di segregazione. Un ex mafioso, colpevole di tanti omicidi, a sua volta vittima della mafia che adesso dice: «Sono contento che è morto Totò Riina, così finalmente si chiude un capitolo».   «Certo, perché noi l’abbiamo seguito e abbiamo sbagliato. Ci siamo fidati delle famiglie che gli stavano intorno, come i Madonia, i Ganci, i Brusca, e lui si faceva forza dell’appoggio di questi. Gli hanno lasciato troppo spazio, e lui ci ha rovinato a tutti. Se invece negli anni Sessanta chi lo voleva togliere

{"_id":{"$oid":"5a207fbb5deb93d55261c784"},"title":["\r\n                    \r\n                        ","\r\n                    \r\n                "," \n                Cosmo: la mia vita divisa in due, tra i figli e le notti in discoteca\n            "," \n                    Cosmo: la mia vita divisa in due, tra i figli e le notti in discoteca\n                "],"text":"","newspaper":"Il Corriere della Sera","resume":["\r\n                    ","\r\n                "],"author":"NP","date":[],"condivisioni":"NP","tags":"NP","link":" http://video.corriere.it/cosmo-mia-vita-divisa-due-figli-notti-discoteca/740318c6-d531-11e7-85e2-6290f9ff2b"}

{"_id":{"$oid":"5a207fbb5deb93d55261c786"},"title":["\r\n                    \r\n                        ","\r\n                    \r\n                "," \n                Dottoressa stuprata in ambulatorio a Catania: «Violentata per un’ora e mezza»\n            "," \n                    Dottoressa stuprata in ambulatorio a Catania: «Viole

In [22]:
for i in range(0, len(collection_testi)):
    f= open(actual_journal[0] + actual_journal[1] + "%s.txt" %str(i) , "w+") 
    f.write(collection_testi[i])
    f.close()


In [23]:
# Routine for cleaning documents
def clean(path):
    with  open(path, 'r') as to_clean:
        one_line = ''
        for string in to_clean.readlines():
            one_line += string.lower()
    to_clean.close()
    
    #Special espressions of html format
    to_replace = ['\\n','\\t','\\r', '\\', '&nbsp'] 
    
    for item in to_replace:
        one_line = one_line.replace(item,' ') 

    cleaned = ' '.join(word for word in one_line.split() if len(word)>1)
    
    # All other special characters
    definitive = re.sub('[^a-zA-Zàéòùè]', ' ', cleaned)
    
    # Just the stopwords remain
    word_tokens = word_tokenize(definitive)
 
    filtered_text = [w for w in word_tokens if not w in stop_words]
    cleaned = ' '.join(word for word in filtered_text if len(word)>1)
    
    with open(path, 'w') as to_clean:
        to_clean.write(cleaned) 
        
    to_clean.close()
    
    return

In [25]:
# Routine for building an interpreter of the stemmed words

def rebuild(parole_intere, parole_stemmizzate):
    vocab = dict()
    
    for i in range(0, len(parole_intere)):
        if parole_stemmizzate[i] not in vocab:
            vocab[parole_stemmizzate[i]] = [parole_intere[i]]
        elif parole_intere[i] not in vocab[parole_stemmizzate[i]]:
            vocab[parole_stemmizzate[i]] =  vocab[parole_stemmizzate[i]] + [parole_intere[i]]
    
    return vocab


In [26]:
nltk.download("stopwords") 
stemmer = SnowballStemmer("italian") 
nltk.download('punkt')
stop_words = set(stopwords.words('italian'))

stemmed_words = []
whole_words = []

for i in range(0, len(collection_testi)):
    file_path = actual_journal[0] + actual_journal[1] + str(i)+ '.txt'
    

    cleaned_text = clean(file_path)

    full_cleaned_text = str("")

    with open(file_path, 'r') as f:
        for line in f.readlines():
            full_cleaned_text = full_cleaned_text + " " + str(line)

    tokens = nltk.word_tokenize(full_cleaned_text)

    stemmed_text = ""

    for token in tokens:
        stemmed_text = stemmed_text + " " + stemmer.stem(token)
        stemmed_words.append(stemmer.stem(token))
    with open(file_path, 'w') as f:
        f.write(stemmed_text)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [27]:
interpreter = rebuild(whole_words, stemmed_words)

In [ ]:
import findspark
findspark.init("/usr/local/spark") # **
import pyspark
from pyspark import SparkContext
sc = SparkContext()
from pyspark.sql.session import SparkSession
spark = SparkSession(sc)

In [101]:
from pyspark.sql import SQLContext, Row
from pyspark.ml.feature import CountVectorizer
from pyspark.mllib.clustering import LDA, LDAModel
from pyspark.mllib.linalg import Vector, Vectors
from pyspark.ml.feature import StopWordsRemover

lista_topic = []
lista_pesi = []

#------ STARTING LDA PROCEDURE ----------
start_LDA_time = time.time()

for news in range(0, len(collection_testi)): 
    
    file_path = actual_journal[0] + actual_journal[1] + str(news)+ '.txt' 

    data = sc.textFile(file_path).zipWithIndex().map( lambda cleanwords_idd: Row(idd = cleanwords_idd[1], cleanwords = cleanwords_idd[0].split(" ")))
    docDF = spark.createDataFrame(data)


    Vector = CountVectorizer(inputCol="cleanwords", outputCol="vectors")
    model = Vector.fit(docDF)
    result = model.transform(docDF)

    corpus = result.select("idd", "vectors").rdd.map(lambda x_y: [x_y[0],Vectors.fromML(x_y[1])]).cache()

    # Cluster the documents into three topics using LDA
    ldaModel = LDA.train(corpus, k = 2, maxIterations = 100,optimizer='online')
    topics = ldaModel.topicsMatrix()
    vocabArray = model.vocabulary

    weight = ldaModel.describeTopics(maxTermsPerTopic = 10)

    wordNumbers = 10  # number of words per topic
    topicIndices = sc.parallelize(ldaModel.describeTopics(maxTermsPerTopic = wordNumbers))
    
    def topic_render(topic):  # specify vector id of words to actual words
        terms = topic[0]
        result = []
        for i in range(wordNumbers):
            term = vocabArray[terms[i]]
            result.append(term)
        return result
    
    
    topics_final = []
    topics_final = topicIndices.map(lambda topic: topic_render(topic)).collect()
    
    i = 0 
    print('---------- NEWS n° %s' %news)    
    for topic in range(len(topics_final)):
        print ("Topic" + str(topic) + ":")
        j = 0
        terms = ''
        for term in topics_final[topic]:
            print (term + '  ' + str(weight[i][1][j]))
            terms = terms + ' ' + term
            lista_pesi.append(weight[i][1][j])
            j += 1
        i += 1
        print ('\n')
        lista_topic.append(terms) # Qui potrebbe esserci qualcosa che non va... gli devo passare le cose in modo da avere 
                                  # confronti con i documenti...
        
        
print("--- The LDA procedure took  %s seconds  ---" % (time.time() - start_LDA_time))

Topic0:
mort  0.00652929037149318
fatt  0.0064331930806742705
voc  0.00610687571924736
riin  0.006056959327563315
sempr  0.006055495552439158
poi  0.006036356865452075
cos  0.006023333916555863
strag  0.006021160873587707
togl  0.005941324168766332
sol  0.005886524037224009


Topic1:
no  0.006158704026831954
luc  0.006138365196156161
togl  0.00613364791759979
vial  0.005855877478125964
buon  0.005846279005918517
destin  0.005820860285853603
mal  0.005760103714181929
stermin  0.005757861320142057
par  0.005718539187046166
chiud  0.005718210755638325


---------- NOTIZIA n° 0
Topic0:
imam  0.007559372703502213
sospett  0.0067591315207869845
spagnol  0.006421775835373217
barcellon  0.006144729004500058
poc  0.006072858062256796
stess  0.005989888308108439
attacc  0.005944038164594036
stat  0.005806250630262205
furgon  0.005720891617121246
prim  0.005709046574374121


Topic1:
abdelbak  0.0065244660914107345
imam  0.006136025936899965
centr  0.006009479160986967
delgad  0.005943034836021196

Topic0:
verbal  0.00784948029467742
sostanz  0.00744073263744507
invit  0.007377610227830498
not  0.0072028058291982505
tant  0.007199385861122614
riusc  0.007119288489391814
grugliasc  0.007057714795247315
propr  0.0070306802224729915
trasport  0.006963184209765078
aggiunt  0.006951015787893991


Topic1:
stat  0.008419189147636131
procur  0.007794688389268658
torin  0.0076505286172164995
uom  0.007602362506862601
enne  0.007452979785562058
fatt  0.00715972384731488
scors  0.0069982708776137196
vai  0.00698601827393333
insult  0.006865751447123298
serviz  0.006848140572523149


---------- NOTIZIA n° 14
Topic0:
soprattutt  0.007036267510269075
aument  0.006947660281976763
morbid  0.006454318394885254
carig  0.006424742793265311
oltre  0.006320349615912927
volp  0.006272152214046633
gabriel  0.00617634257030351
miliard  0.0061738090868162865
sospes  0.006122572741022976
caric  0.006121289178498318


Topic1:
banc  0.008144738833610235
aument  0.0066981597640499045
prospett  0.006677475723

Topic0:
anno  0.02298578264169841
cos  0.022915067944746834
risc  0.022771364290262056
vit  0.02244797416804361
mil  0.022032503547997982
vacanz  0.02092071205006351
certos  0.020901752547552294
settiman  0.020758486504887298
disposizion  0.020533390893197875
escon  0.02042060865387349


Topic1:
addi  0.023149393107443135
tenor  0.0226778380594977
dover  0.022148245480263697
np  0.021765604176545064
trainer  0.020941461396635504
domest  0.020708267001585248
righ  0.02069051112818904
frequent  0.02032779540891134
provved  0.02025305641319789
ora  0.02024797694162068


---------- NOTIZIA n° 28
Topic0:
mil  0.015086868270094861
gip  0.013190084276649965
altro  0.011914598970474545
stat  0.011787849956979837
stupr  0.011314397667064952
mattin  0.010837826697519011
pm  0.0107570384766707
ferm  0.010502941223160976
ora  0.010455460494371481
tren  0.010240567853373893


Topic1:
poi  0.010509368505782591
salvadoregn  0.009988709388968663
gianluc  0.009971947385878779
graz  0.009917187312579968

Topic0:
rowling  0.004076094240388675
laur  0.0039971735618432765
neo  0.003982241918119258
risult  0.0039593141231853155
gest  0.003942531385180118
andrann  0.0038862825818966346
rend  0.003868834097037001
stanford  0.003838361439486458
ben  0.003738986617888238
pied  0.0037025048767294097


Topic1:
fall  0.006819692103499756
rowling  0.006391925224310965
discors  0.00618495009900506
cos  0.0057189233078049325
laur  0.005178963205879355
liber  0.004740924024075029
ide  0.00462761040215727
argoment  0.004616556134829227
vit  0.004616430137697857
giovan  0.004552932215102823


---------- NOTIZIA n° 42
Topic0:
casc  0.015562705092049748
azul  0.013939086667546029
agu  0.013072841651295105
local  0.01114350565540238
senz  0.01086439574504231
villagg  0.010796705936319437
dop  0.01073592062522343
ricostru  0.010565606048029383
nuov  0.010550122327554274
isol  0.010475350592801379


Topic1:
  0.01129385067190341
municip  0.011122744390075125
colp  0.011042146878595976
messic  0.010856333075

Topic0:
dane  0.02037274413826815
assist  0.020019298667255037
bocc  0.019858411360958395
accort  0.019714845008888926
bambin  0.019513515254360472
sub  0.019406752683598587
redatt  0.019397728345573934
mess  0.01918592550402971
verif  0.01902113295449287
tavol  0.01889769999711427


Topic1:
mens  0.022399711802318995
pan  0.021195582059288107
trov  0.020244927198982885
local  0.020175900568455307
commission  0.020075382685784713
alcun  0.019898460084097975
stat  0.019712795796444197
scen  0.019621755626697046
element  0.019532842988534246
piccol  0.019223791968554477


---------- NOTIZIA n° 56
Topic0:
amor  0.016250368268838
complic  0.007723348934856946
sent  0.007689440873927725
vit  0.006695351013700083
form  0.006575800364848069
cresc  0.006496108165355506
esser  0.006457485201237926
cos  0.0062688972704193465
mond  0.006255284504675405
occhi  0.005995273875530824


Topic1:
via  0.006378943772635809
indifes  0.006255180262159309
accec  0.006173241834581619
cos  0.00582353485378169

...

Topic0:
animal  0.01583084369922815
macell  0.012224725992118156
immagin  0.008205259903254599
maltratt  0.00795420876126993
veng  0.0077085660323136145
stord  0.007675828145053602
equality  0.007665989465347494
europe  0.007612389386114564
organizz  0.007447489792577133
gonf  0.007336667355411635


Topic1:
tal  0.008032420915010038
sgozz  0.007352721190065633
derog  0.007350557430355579
matte  0.007329204965633341
testimon  0.007316755750202073
trasform  0.007303100067303172
inutil  0.0072757700124922164
dolor  0.0072655836833915455
stanz  0.007246163854305473
possibil  0.0071974307290686105


---------- NOTIZIA n° 644
Topic0:
ugual  0.008237255071518726
sent  0.007930433159802032
cup  0.007852713842549685
ora  0.0077554593585240925
grass  0.00772864486763662
uscit  0.007306416868388372
puo  0.0072167249284582615
stess  0.007212543846277827
volt  0.007204204680873566
già  0.0070420853543121074


Topic1:
segretar  0.008426105163998101
grass  0.00837068709927832
bersan  0.00825592054133

Topic0:
tratt  0.010860961455473973
pianegg  0.010062871504059418
gioved  0.009864735153567453
pomerigg  0.009860735073052048
trivenet  0.009843147925708671
possibil  0.009837454892112716
citt  0.009788866768228576
diffus  0.009786585206616415
perturb  0.009569915047025661
emil  0.009554076785172777


Topic1:
nev  0.015612421910931899
piogg  0.013214661879663565
nuov  0.012771335666384054
nord  0.012719819854316871
maltemp  0.012496164409605402
perturb  0.011507045272719695
luned  0.011208389699814213
giorn  0.010956382017127822
domen  0.010533426746066816
temporal  0.010481466171515574


---------- NOTIZIA n° 658
Topic0:
raccont  0.011696372102704024
molest  0.011556764707901436
rivel  0.011527357328825559
tre  0.011479739623804138
sess  0.011377309044258708
richiest  0.011231678949292005
stat  0.011189941336343195
sol  0.01110556640939093
mohamed  0.011102614471872597
comport  0.01107675851393587


Topic1:
film  0.011841824300565943
mostr  0.011350859932581442
già  0.0112275777254673

Topic0:
elin  0.00997423585395117
atlet  0.008458211088504034
carrier  0.008298628474879877
stat  0.008250098893305744
torne  0.008150954395407395
internazional  0.007927133072417209
men  0.007883097987453005
sharapov  0.007831309008356963
camp  0.00781356979160757
russ  0.007805915618891446


Topic1:
present  0.0086614878550758
femminil  0.00836364099400929
ucrain  0.008353630138887983
prest  0.0076849709179467535
grand  0.007677533904776889
topless  0.007636930608769085
giust  0.007625100287077695
campioness  0.007615216901001635
modell  0.007597085116091642
sport  0.007570293069605656


---------- NOTIZIA n° 672
Topic0:
sinistr  0.010691312533921478
mess  0.00939521524089451
inter  0.009321462636147736
potr  0.009259614267990868
allen  0.00912467544619631
sfid  0.008907912380305095
higuain  0.008859254015545007
camp  0.008704047592824598
problem  0.008669823600367235
douglas  0.008592494176308791


Topic1:
buffon  0.009621945957463468
napol  0.009429986352486933
ennesim  0.009393302

Topic0:
stat  0.005075359161638075
anni  0.004971910925838627
donn  0.004810581202182816
lavor  0.004722917696609028
chirurg  0.004557340897063796
general  0.004485031346849065
torin  0.004463119686238418
riserv  0.004411689013071336
bosc  0.004370898369605263
medic  0.004350932504803812


Topic1:
concentr  0.004382714762612521
tizian  0.004381134698130415
timor  0.004350037047919743
sal  0.004343374335063218
avant  0.004338850946217312
autorevolezz  0.00425602051110227
stat  0.0042553417209738745
second  0.00423695310764498
organizz  0.004198921925418207
uomin  0.004186623046226462


---------- NOTIZIA n° 686
Topic0:
arriv  0.005551357763878775
ottim  0.005459779694120883
rimand  0.005423396261215516
ten  0.005352744284024908
occasion  0.0052749315810820055
classif  0.0052329047773968865
second  0.0051965250946122525
jorginh  0.005188558203652624
impegn  0.005174250543018988
tre  0.005132644323866739


Topic1:
napol  0.009109111115587208
prim  0.0068084895281248685
temp  0.00637711346

KeyboardInterrupt: 

In [105]:
vectorizer = TfidfVectorizer() # use_idf=True, ngram_range=(1, 3)
 
#------ STARTING LSI PROCEDURE ----------
start_LSI_time = time.time()

X = vectorizer.fit_transform(lista_topic) 

lsa = TruncatedSVD(n_components = 10, n_iter = 1000) # Gauge the components
lsa.fit(X)

terms = vectorizer.get_feature_names()
out = []
for i, comp in enumerate(lsa.components_):
    termsInComp = zip(terms,comp)
    sortedTerms = sorted(termsInComp, key=lambda x: x[1], reverse=True ) [:10]
    
    for term in sortedTerms:
        out.append(term[0])
        
print("--- The LSI procedure took  %s seconds  ---" % (time.time() - start_LSI_time))

# We could sum and/or compare times at this point...

TruncatedSVD(algorithm='randomized', n_components=10, n_iter=1000,
       random_state=None, tol=0.0)

In [ ]:
# Use the interpreter
print(out)
for w in out:
    print(w, interpreter[w])

In [107]:
#  Saving
with open("/root/Desktop/Output News.txt", "w") as output:
    output.write("\n".join(map(lambda x: str(x), out)))